In [14]:
#set openai key in .env.local
import os
import pandas as pd
import openai
import numpy as np
#from openai.embeddings_utils import get_embedding

In [15]:
def estimate_price(n):
    num_tokens = 100 * n
    # per thousend tokens 0.0001 $
    price_per_thousend = 0.0001
    price = round(num_tokens/1000 * price_per_thousend, 5)

    response = input(f"Estimated price {price} $\nProceed? (y/n): ")
    if response.lower() == 'y':
        print("Price accepted...")
        # Continue with your notebook code here

    elif response.lower() == 'n':
        print("Price declined, interrupting...")
        raise KeyboardInterrupt  # Raises a KeyboardInterrupt to stop execution

    else:
        print("Invalid input. interrupting...")
        raise KeyboardInterrupt
        # Prompt the user again or take appropriate action


In [16]:
def set_environment_variables_from_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if line and not line.startswith('#'):  # Ignore empty lines and comments
            key, value = line.split('=', 1)
            os.environ[key.strip()] = value.strip()
file_path = '../../.env.local'
set_environment_variables_from_file(file_path)
openai.api_key = os.getenv('OPENAI_KEY')

In [17]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [27]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'no_product_variants.pkl')
product_data = product_data.reset_index(drop=True)
# product_data = product_data.sample(n=2000, random_state=2).reset_index(drop=True)
display(product_data)

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,40.0,schwarz,02.643.57,6590678,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0
1,551622,4054452768212,elegante Pumps Glattleder schwarz,38.0,schwarz,05.160.37,6590677,315573,w,PT,...,Pumps,Damenschuhe,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0
2,547193,4054452768427,Slipper Glattleder schwarz,42.0,schwarz,04.443.27,6590677,315576,w,SK,...,Halbschuhe,Damenschuhe,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0
3,549776,4059701687894,Sneaker low Rauleder blau,39.0,blau,06.968.46,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat schwarz,37.5,schwarz,06.102.67,6590678,315574,w,SK,...,Ballerinas,Damenschuhe,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,35.0,blauw,24.764.36,6590677,315568,w,SK,...,Sandalen,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,40.5,pink,26.090.21,6590678,315590,w,PT,...,Halbschuhe,Damenschuhe,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0
3294,10782704,4251234499207,Shopper ANDIE blau,,blau,921453,363013,363017,w,DE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3295,10679703,4066558303617,Elegante pumps Glad leer wit,35.0,wit,21.450.60,6590677,315573,w,PT,...,Pumps,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0


In [28]:
def combine_columns(row):
    return (
            "ID: " + str(row.product__id) + "; " +
            "product_number: " + str(row.productNumber) + "; " +
            "product_name: " + row.productName + "; " +
            # "size: " + str(row.productSizeEu) + "; " +
            "color: " + str(row.productColorName) + "; " +
            "article number: " + str(row.articleNumber) + "; " +
            "brand: " + str(row.brand__id) + "; " +
            "main category: " + str(row.mainCategory__id) + "; " +
            "gender: " + str(row.gender) + "; " +
            "line: " + str(row.line) + "; " +
            "price: " + str(row.b2cPrice) + "; " +
            "material: " + str(row.material) + "; " +
            "material PM: " + str(row.materialPm) + "; "
    )

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

def get_embeddings(texts, model="text-embedding-ada-002"):
    def chunk_list(lst, chunk_size):
        for i in range(0, len(lst), chunk_size):
            yield lst[i:i + chunk_size]

    # Split the texts into chunks of 1000
    text_chunks = list(chunk_list(texts, 1000))
    all_embeddings = []

    # Process each chunk separately
    for chunk in text_chunks:
        embeddings = openai.Embedding.create(input=chunk, model=model).data
        all_embeddings.extend(embeddings)

    return all_embeddings


def get_list_from_embeddings(embedding_result):
    list = []
    for entry in embedding_result:
        list.append(entry.embedding)
    return list

In [30]:
product_embeddings_path = "openai/product_embeddings.pkl"
recreate_embeddings = False
if not os.path.exists(basepath + product_embeddings_path) or recreate_embeddings:
    product_data_embeddings = product_data[['product__id']].copy()
    product_data_embeddings["combined"] = product_data.apply(lambda row: combine_columns(row), axis=1)
    # print(product_data_embeddings.combined.tolist())
    estimate_price(product_data_embeddings.shape[0])
    # embed each row separately
    # product_data_embeddings['embedding'] = product_data_embeddings.combined.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
    # batch embed
    product_data_embeddings['embedding'] = get_list_from_embeddings(get_embeddings(product_data_embeddings.combined.tolist()))
    product_data_embeddings.to_pickle(basepath + product_embeddings_path)
else:
    product_data_embeddings = pd.read_pickle(basepath + product_embeddings_path)
display(product_data_embeddings)

,product__id,combined,embedding
0,557559,ID: 557559; product_number: 4058394021466; product_name: sportliche Ballerinas Glattleder schwarz; color: schwarz; article number: 02.643.57; brand: 6590678; main category: 315571; gender: w; line: F-S|H-W; price: 99.95; material: Glattleder; material PM: Foulardcalf;,"[-0.00733334943652153, 0.00807138066738844, -0.002195643726736307, -0.023737777024507523, -0.013204054906964302, 0.004988422151654959, -0.024932047352194786, -0.011023507453501225, -0.005900897551327944, -0.05007879436016083, 0.02895767241716385, 0.009077787399291992, -0.023080257698893547, -0.0027843916323035955, -0.0007019686163403094, 0.023509658873081207, 0.015914643183350563, 0.010808806866407394, 0.01874599978327751, 0.0015741876559332013, 0.012553244829177856, 0.021859152242541313, -0.004042400047183037, -0.0006562609924003482, -0.01283503882586956, -0.0025914970319718122, 0.018544718623161316, -0.02662280946969986, -0.019819501787424088, -0.01051359437406063, -0.016894211992621422, -0.005921025760471821, -0.03177561238408089, -0.008386721834540367, -0.014317811466753483, 0.008621550165116787, -0.00390485767275095, -0.005880769342184067, 0.002791101112961769, 0.01603541150689125, 0.010956413112580776, 0.017605407163500786, 0.013821317814290524, 0.010835644789040089, -0.0019071404822170734, 0.0016530228313058615, -0.0010022132191807032, -8.63832319737412e-05, -0.009460221976041794, 0.019591381773352623, 0.006249784957617521, 0.05399706959724426, -0.00042520678834989667, -0.005525172222405672, 0.006944205611944199, 0.0070649744011461735, 0.03864601254463196, 0.010453210212290287, 0.018075061962008476, -0.008775865659117699, -0.023294959217309952, 0.0071052308194339275, -0.025965290144085884, -0.008608131669461727, -0.02699853479862213, -0.02844776026904583, -0.03306381031870842, 0.01647823117673397, -0.009983553551137447, -0.006826791446655989, -0.0078097153455019, 0.018128737807273865, 0.019336426630616188, -0.01514977402985096, 0.045167528092861176, 0.0008747350657358766, -0.02147000841796398, 0.02196650207042694, 0.007467537187039852, -0.004488573409616947, 0.02839408442378044, -0.0026485268026590347, -0.005719744134694338, -0.009614537470042706, 0.017202842980623245, -0.004884426947683096, -0.006524869706481695, 0.03072894923388958, -0.03666003793478012, -0.0011615609982982278, 0.01234525442123413, 0.01745779998600483, -0.006112243048846722, -0.0006004892638884485, -0.011600513942539692, 0.003243983956053853, -0.0173504501581192, 0.022610602900385857, 0.002046360168606043, -0.008306208997964859, ...]"
1,551622,ID: 551622; product_number: 4054452768212; product_name: elegante Pumps Glattleder schwarz; color: schwarz; article number: 05.160.37; brand: 6590677; main category: 315573; gender: w; line: F-S|H-W; price: 89.95; material: Glattleder; material PM: Kid;,"[0.0013092956505715847, 0.009199109859764576, -0.008940073661506176, -0.036849573254585266, 0.00795042235404253, 0.0061006369069218636, -0.02217085100710392, -0.00579510722309351, -0.022237271070480347, -0.035733725875616074, 0.020776040852069855, 0.006170377600938082, -0.031164059415459633, -0.012579865753650665, -0.003566730534657836, 0.040011145174503326, 0.008262594230473042, 0.011377671733498573, 0.005077775567770004, 0.007206522859632969, -0.0036763229873031378, 0.022755343466997147, 0.001992587000131607, 0.0011847589630633593, -0.0018946181517094374, -0.015396054834127426, 0.018398219719529152, -0.020869027823209763, -0.02159964293241501, -0.02217085100710392, -0.007060399744659662, -0.014904550276696682, -0.031164059415459633, -0.009159257635474205, -0.016671311110258102, 0.004433506168425083, -0.01184925064444542, -0.008813876658678055, 0.013576159253716469, 0.010779895819723606, 0.014904550276696682, -0.0008020162349566817, 0.02582392655313015, 0.001756797544658184, -0.013569517061114311, 0.00194443273358047, -0.0011100369738414884, 0.009159257635474205, -0.0041578649543225765, 0.01485141459852457, -0.006974054500460625, 0.053587306290864944, -0.0

In [31]:
product_data_embeddings.iloc[0].embedding

[-0.00733334943652153,
 0.00807138066738844,
 -0.002195643726736307,
 -0.023737777024507523,
 -0.013204054906964302,
 0.004988422151654959,
 -0.024932047352194786,
 -0.011023507453501225,
 -0.005900897551327944,
 -0.05007879436016083,
 0.02895767241716385,
 0.009077787399291992,
 -0.023080257698893547,
 -0.0027843916323035955,
 -0.0007019686163403094,
 0.023509658873081207,
 0.015914643183350563,
 0.010808806866407394,
 0.01874599978327751,
 0.0015741876559332013,
 0.012553244829177856,
 0.021859152242541313,
 -0.004042400047183037,
 -0.0006562609924003482,
 -0.01283503882586956,
 -0.0025914970319718122,
 0.018544718623161316,
 -0.02662280946969986,
 -0.019819501787424088,
 -0.01051359437406063,
 -0.016894211992621422,
 -0.005921025760471821,
 -0.03177561238408089,
 -0.008386721834540367,
 -0.014317811466753483,
 0.008621550165116787,
 -0.00390485767275095,
 -0.005880769342184067,
 0.002791101112961769,
 0.01603541150689125,
 0.010956413112580776,
 0.017605407163500786,
 0.013821317814

In [32]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_n_nearest_neighbors(query_embedding, neighbors_embedding, n = 5):
    similarities = [cosine_similarity(query_embedding, neighbor) for neighbor in neighbors_embedding]
    indices = np.flip(np.argsort(similarities)[-n:])
    scores = [similarities[i] for i in indices]
    # Get the n neighbors with the highest similarity values
    n_neighbors = [neighbors_embedding[i] for i in indices]

    return indices, scores


indices, scores = get_n_nearest_neighbors(product_data_embeddings.embedding[0], product_data_embeddings.embedding)
display(indices, scores)

array([   0,   33, 3039, 1237,    8])

[1.0,
 0.9953282182374915,
 0.9855057298396938,
 0.985211016535581,
 0.9817211262258722]

In [33]:
pd.set_option('display.max_colwidth', None)
for data in product_data_embeddings.iloc[indices].combined:
    print(data.replace('; ', ";\n"), "\n")

ID: 557559;
product_number: 4058394021466;
product_name: sportliche Ballerinas Glattleder schwarz;
color: schwarz;
article number: 02.643.57;
brand: 6590678;
main category: 315571;
gender: w;
line: F-S|H-W;
price: 99.95;
material: Glattleder;
material PM: Foulardcalf;
 

ID: 589474;
product_number: 4056068820575;
product_name: Sportliche Ballerina Glattleder schwarz;
color: schwarz;
article number: 02.020.57;
brand: 6590678;
main category: 315571;
gender: w;
line: F-S|H-W;
price: 89.95;
material: Glattleder;
material PM: Foulardcalf;
 

ID: 10231215;
product_number: 4065171089236;
product_name: Sportliche Ballerina Glattleder schwarz;
color: schwarz;
article number: 24.120.27;
brand: 6590677;
main category: 315574;
gender: w;
line: F-S;
price: 89.95;
material: Glattleder;
material PM: Lammnappa;
 

ID: 6112332;
product_number: 4064032647653;
product_name: Sportliche Ballerina Glattleder Schwarz;
color: Schwarz;
article number: 74.165.27;
brand: 6590677;
main category: 315571;
gender: w

In [34]:
product_data.iloc[indices]

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,40.0,schwarz,02.643.57,6590678,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0
33,589474,4056068820575,Sportliche Ballerina Glattleder schwarz,35.0,schwarz,02.020.57,6590678,315571,w,SK,...,Ballerinas,Damenschuhe,bis 3 cm,35.0,2.5,3.0,0.0,Gummi,0.0,0.0
3039,10231215,4065171089236,Sportliche Ballerina Glattleder schwarz,40.0,schwarz,24.120.27,6590677,315574,w,SK,...,Ballerinas,Damenschuhe,bis 3 cm,40.0,6.5,11.0,0.0,TPU,0.0,0.0
1237,6112332,4064032647653,Sportliche Ballerina Glattleder Schwarz,35.5,Schwarz,74.165.27,6590677,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,35.5,3.0,4.0,0.0,Gummi,0.0,0.0
8,548618,4058394180835,sportliche Ballerinas Rauleder schwarz,42.0,schwarz,02.643.47,6590678,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,42.0,8.0,14.0,0.0,Gummi,0.0,0.0


In [35]:
def get_best_recommendations_multiple(query_embeddings, neighbors_embeddings, n = 5):
    all_results = {}
    for query_embedding in query_embeddings:
        indices, scores = get_n_nearest_neighbors(query_embedding, neighbors_embeddings, n)
        result_dict = {index: score for index, score in zip(indices, scores)}
        for key, value in result_dict.items():
            if key in all_results:
                all_results[key] = max(all_results[key], value)
            else:
                all_results[key] = value
    return dict(sorted(all_results.items(), key=lambda item: item[1], reverse=True)[:n])


best_items = get_best_recommendations_multiple([product_data_embeddings.embedding[0], product_data_embeddings.embedding[1]], product_data_embeddings.embedding)

print(best_items)

{1: 1.0000000000000002, 0: 1.0, 33: 0.9953282182374915, 3039: 0.9855057298396938, 1237: 0.985211016535581}


In [43]:
def evaluate_coverage(embeddings_df, n=5):
    embeddings = embeddings_df.tolist()
    recommended_items = set()  # Set to store unique recommended items
    total_items = len(embeddings)
    scores_list = []

    for i, query_embedding in enumerate(embeddings):
        # Exclude the current query embedding from the neighbors
        neighbors_embeddings = embeddings[:i] + embeddings[i+1:]

        indices, scores = get_n_nearest_neighbors(query_embedding, neighbors_embeddings, n)
        recommended_items.update(indices)
        scores_list.extend(scores)
        if i % 100 == 0:
            print(i)

    coverage_score = len(recommended_items) / (total_items - 1)  # Adjusting total_items to account for removal

    return coverage_score, scores_list

# Example usage
coverage_score, confidence_scores = evaluate_coverage(product_data_embeddings.embedding)
print(f"Coverage Score: {coverage_score}")
print(f"Confidence Score: {np.mean(confidence_scores)}; Min Score: {np.min(confidence_scores)}; Max Score: {np.max(confidence_scores)}")


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
Coverage Score: 0.9736043689320388
Confidence Score: 0.9796308191166315; Min Score: 0.8994425934897784; Max Score: 0.9994907391853894
